# FIT5202 Group 1 Phase 3
Alexis Illig  
Robert Allan  
Nathan Davis  
Enzo Reyes  
Ninette Macalalad  
Enviroment: Jupyter notebook with spylon-kernel, scala version 2.4.3  

## Initialize Spark

Set memory parameters before initializing Spark:

In [1]:
%%init_spark
launcher.num_executors = 1
launcher.executor_cores = 6
launcher.driver_memory = '10g'

In [2]:
// initialize scala
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://1dfa34b9cd3c:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559910304267)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6f80083f


In [3]:
// set error log level to less verbose
sc.setLogLevel("ERROR")

## Load Libraries

In [4]:
import java.time.LocalDateTime
import java.io.{File, PrintWriter}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.rand
import org.apache.spark.ml.linalg.{Vector}
import org.apache.spark.ml.feature.{HashingTF, IDF, VectorAssembler, StringIndexer, OneHotEncoderEstimator}
import org.apache.spark.ml.classification.{LogisticRegression, NaiveBayes, RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.classification.{DecisionTreeClassifier, DecisionTreeClassificationModel, MultilayerPerceptronClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._

import java.time.LocalDateTime
import java.io.{File, PrintWriter}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.rand
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.feature.{HashingTF, IDF, VectorAssembler, StringIndexer, OneHotEncoderEstimator}
import org.apache.spark.ml.classification.{LogisticRegression, NaiveBayes, RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.classification.{DecisionTreeClassifier, DecisionTreeClassificationModel, MultilayerPerceptronClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._


In [5]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 25


## User-defined Functions

In [6]:
// Function to search a directory and return a list of files based on filetype
def getListOfFiles(dir: File, extensions: List[String]): List[File] = {
    dir.listFiles.filter(_.isFile).toList.filter { file =>
        extensions.exists(file.getName.endsWith(_))
    }
}
//Source: https://alvinalexander.com/scala/how-to-list-files-in-directory-filter-names-scala

getListOfFiles: (dir: java.io.File, extensions: List[String])List[java.io.File]


In [7]:
// Function to return the metrics for each label in a dataframe 
def metrics_by_label(trsum: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary): org.apache.spark.sql.DataFrame = {
    // false positives
    var fp = sc.parallelize(trsum.falsePositiveRateByLabel.zipWithIndex).toDF("False Positive Rate", "Label")
    // true positives
    var tp = sc.parallelize(trsum.truePositiveRateByLabel.zipWithIndex).toDF("True Positive Rate", "Label") 
    // precision
    var pc = sc.parallelize(trsum.precisionByLabel.zipWithIndex).toDF("Precision", "Label") 
     // recall
    var rc = sc.parallelize(trsum.recallByLabel.zipWithIndex).toDF("Recall", "Label")   
    // f-measure
    var fm = sc.parallelize(trsum.fMeasureByLabel.zipWithIndex).toDF("F-Measure", "Label")    
    
    // merge dataframes
    var metrics1 = fp.join(tp, "Label").join(pc, "Label").join(rc, "Label").join(fm, "Label")

    return metrics1
}

metrics_by_label: (trsum: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary)org.apache.spark.sql.DataFrame


In [8]:
// Function to return the total metrics in dataframe
def metrics_all(trsum: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary): org.apache.spark.sql.DataFrame = {
    var temp_arr = Array[Double](trsum.accuracy, 
                                 trsum.weightedFalsePositiveRate, 
                                 trsum.weightedTruePositiveRate, 
                                 trsum.weightedFMeasure, 
                                 trsum.weightedPrecision, 
                                 trsum.weightedRecall)
    val lab_arr = Array[String]("Accuracy",
                                "False Positive Rate",
                                "True Positive Rate",
                                "F-Measure", 
                                "Precision", 
                                "Recall")
    var metrics2 = sc.parallelize(lab_arr zip temp_arr).toDF("Metric","Values")
    return metrics2
}

metrics_all: (trsum: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary)org.apache.spark.sql.DataFrame


## Load Files
ATTENTION: Please refer to the Phase 3 report for a link to the "output_dir2" folder referenced in the cell below.  
You will need to download this folder into the same directroy as this ipynb file.

In [9]:
// get json file name
val okFileExtensions = List("json")
val myFiles = getListOfFiles(new File("./output_dir2"), okFileExtensions)

okFileExtensions: List[String] = List(json)
myFiles: List[java.io.File] = List(./output_dir2/part-00000-1be6ed82-1dd3-437f-ac0a-7b6b03ddc15c-c000.json)


In [10]:
// read the json file into a spark dataframe
val sqlContext = new org.apache.spark.sql.SQLContext(sc)    // create an SQL Context object
var df_both = sqlContext.read.json(myFiles(0).toString())

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@733e4845
df_both: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 3 more fields]


In [11]:
// rename column if need be
if (df_both.columns.contains("words_stopped2")){
    df_both = df_both.withColumnRenamed("words_stopped2","words_stopped")
}

In [12]:
// shuffle the dataset rows in case of class clustering
val df_both2 = df_both.orderBy(rand(seed=10L))    

df_both2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, parent_score: bigint ... 3 more fields]


In [13]:
// view the data
df_both.show(5)

+-------+------------+-----+------------+--------------------+
|     id|parent_score|score|subreddit_id|       words_stopped|
+-------+------------+-----+------------+--------------------+
|c092j8m|       -7526| 9582|    t5_2qh2p|[twitter, comment...|
|c4imcva|        3841| 9531|    t5_2qh1i|[well, exactly, s...|
|c0s4nfi|         621| 8545|    t5_2qh1i|[soviet, russia, ...|
|c4ini33|        4651| 7430|    t5_2qh1i|       [runin, yolo]|
|c4imgel|        4251| 7173|    t5_2qh1i|[step, motherfucker]|
+-------+------------+-----+------------+--------------------+
only showing top 5 rows



## Feature Engineering

### TF
Calculate Term Frequencies (TF)

In [14]:
val tfModel = new HashingTF().setInputCol("words_stopped").setOutputCol("TF").setNumFeatures(200)

tfModel: org.apache.spark.ml.feature.HashingTF = hashingTF_c18490e1469d


In [15]:
val df_both3 = tfModel.transform(df_both2)

df_both3: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 4 more fields]


### IDF
Calculate the Inverse Document Frequency (IDF) vectors

In [16]:
// define the idf vectorizer model
var idf = new IDF().setInputCol("TF").setOutputCol("TF-IDF").setMinDocFreq(2)

idf: org.apache.spark.ml.feature.IDF = idf_a95f8a33f472


In [17]:
// compute the idf vectors
val idfModel = idf.fit(df_both3)

idfModel: org.apache.spark.ml.feature.IDFModel = idf_a95f8a33f472


In [18]:
// scale the tf's by the idf's
val df_both3b = idfModel.transform(df_both3)

df_both3b: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 5 more fields]


In [19]:
// view data
df_both3b.show(2)

+-------+------------+-----+------------+--------------------+--------------------+--------------------+
|     id|parent_score|score|subreddit_id|       words_stopped|                  TF|              TF-IDF|
+-------+------------+-----+------------+--------------------+--------------------+--------------------+
|c6m07b0|         514|  238|    t5_2qh33|     [dolphinplasty]|   (200,[117],[1.0])|(200,[117],[3.364...|
|c3nxcbf|          22|   82|    t5_2qzb6|[voice, venture, ...|(200,[19,91,101,1...|(200,[19,91,101,1...|
+-------+------------+-----+------------+--------------------+--------------------+--------------------+
only showing top 2 rows



### Categorize the Scores
Create classes by binning the scores

In [20]:
val col_id = col("score")
// var col_id = col("parent_score")  // sanity check
var df_both4 = df_both3b.withColumn("label", 
                               when(col_id < -1000  , 1)
                               .when(col_id >= -1000 && col_id < -100 , 2)
                               .when(col_id >= -100 && col_id < 0, 3)
                               .when(col_id >= 0 && col_id <= 100, 4)
                               .when(col_id > 100 && col_id <= 1000,5)
                               .when(col_id > 1000, 6)
                               .otherwise(-1))

col_id: org.apache.spark.sql.Column = score
df_both4: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 6 more fields]


In [21]:
// view the data
df_both4.show(5)

+-------+------------+-----+------------+--------------------+--------------------+--------------------+-----+
|     id|parent_score|score|subreddit_id|       words_stopped|                  TF|              TF-IDF|label|
+-------+------------+-----+------------+--------------------+--------------------+--------------------+-----+
|c6m07b0|         514|  238|    t5_2qh33|     [dolphinplasty]|   (200,[117],[1.0])|(200,[117],[3.364...|    5|
|c3nxcbf|          22|   82|    t5_2qzb6|[voice, venture, ...|(200,[19,91,101,1...|(200,[19,91,101,1...|    4|
|cnm811v|          76|  -19|    t5_2qh61|[well, making, re...|(200,[0,1,88,103,...|(200,[0,1,88,103,...|    3|
|c3ouklj|          88|  120|    t5_2qh03|[install, norton,...|(200,[58,94,157],...|(200,[58,94,157],...|    5|
|c40sx8i|           7|   -9|    t5_2qh4i|[personal, opinio...|(200,[2,13,38,55,...|(200,[2,13,38,55,...|    3|
+-------+------------+-----+------------+--------------------+--------------------+--------------------+-----+
o

### Dimension Reduction
Create two combinations of the tf-idf vectors and other features for model testing

#### Combination 1
Add parent_score as an extra dimension to the tf-idf vectors and store as a new column

In [22]:
// define vector assembler
val assembler = new VectorAssembler()
    .setInputCols(Array("parent_score", "TF-IDF"))
    .setOutputCol("comboFeatures1")

// apply assembler to the data
df_both4 = assembler.transform(df_both4)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_0688f9c7d26d
df_both4: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 7 more fields]


#### Combination 2
Convert subreddit_id from strings to numerical indices

In [23]:
// define indexer
val indexer = new StringIndexer()
  .setInputCol("subreddit_id")
  .setOutputCol("subreddit_id_int")

// apply indexer to the data
df_both4 = indexer.fit(df_both4).transform(df_both4)

indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_f8f4fe16021d
df_both4: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 8 more fields]


Apply One-Hot Encoding to the indexed subreddit id's since that column is linear

In [24]:
// define one-hot encoder
val encoder = new OneHotEncoderEstimator()
  .setInputCols(Array("subreddit_id_int"))
  .setOutputCols(Array("subreddit_idVec"))

// apply one-hot encoding to the data
df_both4 = encoder.fit(df_both4).transform(df_both4)

encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_4bd46c16ca13
df_both4: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 9 more fields]


Add the one-hot encoded subreddit ids as an extra dimension to the tf-idf vectors and store as a new column

In [25]:
// define vector assembler
val assembler = new VectorAssembler()
    .setInputCols(Array("parent_score", "TF-IDF","subreddit_idVec")) //,"subreddit_id_int" 
    .setOutputCol("comboFeatures2")

// apply assembler to the data
df_both4 = assembler.transform(df_both4)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_960c0a403115
df_both4: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 10 more fields]


In [26]:
// define the one-hot encoded dataframe as new variable
val df_workingSet = df_both4

df_workingSet: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 10 more fields]


## Feature Testing
Test multiple sets of feature vectors via logistic regression in order to determine the most suitable set of features for selection.  
See the report for discussion of logistic regression as the choice model for feature testing.

In [27]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 26


In [28]:
// define an 'empty' dataframe in which to collect metric results from each model
var results =  List(("Log Reg 1", 0.0), 
                ("Log Reg 2", 0.0),
                ("Log Reg 3", 0.0),
                ("Log Reg 4", 0.0)).toDF("Model", "F-Measure")

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [30]:
results = results.withColumn("F-Measure", when(col("Model").equalTo("Log Reg 1"), 
                                               0.25).otherwise(col("F-Measure"))) 
results.show(false)

+---------+---------+
|Model    |F-Measure|
+---------+---------+
|Log Reg 1|0.25     |
|Log Reg 2|0.0      |
|Log Reg 3|0.0      |
|Log Reg 4|0.0      |
+---------+---------+



results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


### Logistic Regression 1
Use the term frequencies only as the modelling data

In [32]:
// define logistic regression model
val lr = new LogisticRegression()
    .setMaxIter(100)  
    .setRegParam(1e-5)
    .setElasticNetParam(0)
    .setLabelCol("label")
    .setFeaturesCol("TF")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_0a4bdb773742


In [33]:
// train the model
val lrModel = lr.fit(df_workingSet)

lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_0a4bdb773742, numClasses = 7, numFeatures = 200


In [34]:
// get metrics
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.LogisticRegressionTrainingSummaryImpl@47cc0191


In [35]:
// view metrics by label
metrics_by_label(trainingSummary).show()

+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|Label| False Positive Rate|  True Positive Rate|          Precision|              Recall|           F-Measure|
+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|    1|4.644463721835843E-6|1.115697869017070...|0.05263157894736842|1.115697869017070...| 2.22667557336896E-4|
|    3|6.705818329064822E-5|1.060349199819612...| 0.2852233676975945|1.060349199819612...|2.119910299699126...|
|    5|9.618969231257178E-6|1.579820427078122E-4|0.13953488372093023|1.579820427078122E-4|3.156067539845352...|
|    4|0.017070099393040605|0.024720710576786886| 0.3705172367753478|0.024720710576786886|0.046349038271234815|
|    2|  0.9755509262650316|  0.9857579951165537| 0.5000199512035564|  0.9857579951165537|  0.6634890036394443|
|    0|                 0.0|                 0.0|                0.0|                 0.0|              

In [36]:
// view metrics total
metrics_all(trainingSummary).show()

+-------------------+-------------------+
|             Metric|             Values|
+-------------------+-------------------+
|           Accuracy| 0.4975007368945398|
|False Positive Rate| 0.4902030841688557|
| True Positive Rate| 0.4975007368945397|
|          F-Measure| 0.3434718325082306|
|          Precision|0.41475334214000564|
|             Recall| 0.4975007368945397|
+-------------------+-------------------+



In [37]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Log Reg 1"), 
                                               metrics_all(trainingSummary).collect()(3)(1)
                                              ).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [39]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 29


### Logistic Regression 2
Use the original tf-idf vectors as the modelling data

In [40]:
// define logistic regression model
val lr = new LogisticRegression()
    .setMaxIter(100)  
    .setRegParam(1e-5)
    .setElasticNetParam(0)
    .setLabelCol("label")
    .setFeaturesCol("TF-IDF")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_c6cbf1f25c7f


In [41]:
// train the model
val lrModel = lr.fit(df_workingSet)

lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_c6cbf1f25c7f, numClasses = 7, numFeatures = 200


In [42]:
// get metrics
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.LogisticRegressionTrainingSummaryImpl@73f8a374


In [43]:
// view metrics by label
metrics_by_label(trainingSummary).show()

+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|Label| False Positive Rate|  True Positive Rate|          Precision|              Recall|           F-Measure|
+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|    1|4.644463721835843E-6|1.115697869017070...|0.05263157894736842|1.115697869017070...| 2.22667557336896E-4|
|    3|6.705818329064822E-5|1.060349199819612...| 0.2852233676975945|1.060349199819612...|2.119910299699126...|
|    5|9.618969231257178E-6|1.579820427078122E-4|0.13953488372093023|1.579820427078122E-4|3.156067539845352...|
|    4|0.017070099393040605|0.024720710576786886| 0.3705172367753478|0.024720710576786886|0.046349038271234815|
|    2|  0.9755509262650316|  0.9857579951165537| 0.5000199512035564|  0.9857579951165537|  0.6634890036394443|
|    0|                 0.0|                 0.0|                0.0|                 0.0|              

In [44]:
// view metrics total
metrics_all(trainingSummary).show()

+-------------------+-------------------+
|             Metric|             Values|
+-------------------+-------------------+
|           Accuracy| 0.4975007368945398|
|False Positive Rate| 0.4902030841688557|
| True Positive Rate| 0.4975007368945397|
|          F-Measure| 0.3434718325082306|
|          Precision|0.41475334214000564|
|             Recall| 0.4975007368945397|
+-------------------+-------------------+



In [45]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Log Reg 2"), 
                                               metrics_all(trainingSummary).collect()(3)(1)
                                              ).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [46]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 32


### Logistic Regression 3
Use the first combination tf-idf vectors as the input data

In [47]:
// define logistic regression model
val lr = new LogisticRegression()
    .setMaxIter(100)  
    .setRegParam(1e-5)
    .setElasticNetParam(0)
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures1")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_9ccd0df687f0


In [48]:
// train the model
val lrModel = lr.fit(df_workingSet)

lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_9ccd0df687f0, numClasses = 7, numFeatures = 201


In [49]:
// get metrics
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.LogisticRegressionTrainingSummaryImpl@468bf15c


In [50]:
// view metrics by label
metrics_by_label(trainingSummary).show()

+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|Label| False Positive Rate|  True Positive Rate|          Precision|              Recall|           F-Measure|
+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|    1|4.902489484160057E-6|1.115697869017070...|               0.05|1.115697869017070...|2.226427696760547...|
|    3|1.631319266589807...|2.836114727228362E-4|0.30494505494505497|2.836114727228362E-4|5.666958949008857E-4|
|    5|0.001405149424187...| 0.08612654361620896|0.37701705855232825| 0.08612654361620896| 0.14022076947808382|
|    4|  0.1246163961586755|  0.5494552286204737| 0.6418437576093087|  0.5494552286204737|  0.5920670002279734|
|    2|  0.5644677081375176|   0.937830251076211| 0.6218380324536302|   0.937830251076211|  0.7478237831250572|
|    0|                 0.0|                 0.0|                0.0|                 0.0|              

In [51]:
// view metrics total
metrics_all(trainingSummary).show()

+-------------------+------------------+
|             Metric|            Values|
+-------------------+------------------+
|           Accuracy|0.6261780980784107|
|False Positive Rate|0.3168353862149347|
| True Positive Rate|0.6261780980784106|
|          F-Measure|0.5445658925646598|
|          Precision|0.5600473537533193|
|             Recall|0.6261780980784106|
+-------------------+------------------+



In [52]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Log Reg 3"), 
                                               metrics_all(trainingSummary).collect()(3)(1)
                                              ).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [53]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 35


### Logistic Regression 4
Use the second combination tf-idf vectors as the input data

In [54]:
// define logistic regression model
val lr = new LogisticRegression()
    .setMaxIter(100)  /// upped to 1000
    .setRegParam(1e-5)
    .setElasticNetParam(0)
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures2")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_b08b7f4fe890


In [55]:
// train the model
val lrModel = lr.fit(df_workingSet)

lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_b08b7f4fe890, numClasses = 7, numFeatures = 6093


In [56]:
// get metrics
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.LogisticRegressionTrainingSummaryImpl@19f28c24


In [57]:
// view metrics by label
metrics_by_label(trainingSummary).show()

+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|Label| False Positive Rate|  True Positive Rate|          Precision|              Recall|           F-Measure|
+-----+--------------------+--------------------+-------------------+--------------------+--------------------+
|    1|5.934592533456910...|0.001561977016623...| 0.3783783783783784|0.001561977016623...|0.003111111111111111|
|    3|0.005096099534977...| 0.01756219331315689| 0.4651485416525682| 0.01756219331315689| 0.03384647581295434|
|    5|0.001348475497365...| 0.09086600489744333|0.39951377633711505| 0.09086600489744333| 0.14805757556256302|
|    4| 0.15797454275291062|   0.629426295296286| 0.6182349842768672|   0.629426295296286|  0.6237804477014395|
|    2| 0.47038395167143626|   0.923839767482541| 0.6603065605194025|   0.923839767482541|  0.7701529190259158|
|    0|                 0.0|                 0.0|                0.0|                 0.0|              

In [58]:
// view metrics total
metrics_all(trainingSummary).show()

+-------------------+------------------+
|             Metric|            Values|
+-------------------+------------------+
|           Accuracy|0.6458609695601415|
|False Positive Rate|0.2806699736093649|
| True Positive Rate|0.6458609695601416|
|          F-Measure|0.5716269370365401|
|          Precision|0.6056193435074274|
|             Recall|0.6458609695601416|
+-------------------+------------------+



In [59]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Log Reg 4"), 
                                               metrics_all(trainingSummary).collect()(3)(1)
                                              ).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [60]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 38


### Feature Selection Results

In [61]:
// view the model results
results.show(false)

+---------+------------------+
|Model    |F-Measure         |
+---------+------------------+
|Log Reg 1|0.3434718325082306|
|Log Reg 2|0.3434718325082306|
|Log Reg 3|0.5445658925646598|
|Log Reg 4|0.5716269370365401|
+---------+------------------+



* the feature sets without expanded dimensions (1 and 2) performed similiarly due to using such a low number of features (200)
* the feature vector sets with extra dimensions (3 and 4) faired better in both accuracy and f-score than than the unexpmanded sets
* the second combination of tf-idf vectors with extra dimensions of parent comment score and subreddit id resulted in the best accuracy and f-score  

However,  
* the second combination only faired better than the first combination by 0.01% f-score, the most relevant metric
* the second combination also took significantly more time to run than the first combination  
* the performance improvement of the second combination over the first is not significant enough to justify the increase in processing time  

**Feature set combination 1 is therefore chosen as the most suitable feature set**

## Model Comparison
**Models tested:**  
Logistic Regression with increased iterations  
Random Forest   
Multilayer Perceptron Classifier (neural net)  
Decision Tree  
Naive Bayes  

In [62]:
// define an 'empty' dataframe in which to collect metric results from each model
var results =  List(("Logistic Regression", 0.0), 
                ("Random Forest", 0.0),
                ("Multilayer Perceptron Classifier", 0.0),
                ("Decision Tree", 0.0),
                ("Naive Bayes", 0.0)).toDF("Model", "F-Measure")

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


### Logistic Regression 
This model differs from those in the feature selection section by increasing the iteration parameter from 100 to 1000 iterations

In [63]:
// define logistic regression model
val lr = new LogisticRegression()
    .setMaxIter(1000)  
    .setRegParam(1e-5)
    .setElasticNetParam(0)
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures1")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_051f1d90ea0f


In [64]:
// train the model
val lrModel = lr.fit(df_workingSet)

lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_051f1d90ea0f, numClasses = 7, numFeatures = 201


In [65]:
// get metrics
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.classification.LogisticRegressionTrainingSummary = org.apache.spark.ml.classification.LogisticRegressionTrainingSummaryImpl@14a81317


In [66]:
// view metrics by label
metrics_by_label(trainingSummary).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|Label| False Positive Rate|  True Positive Rate|           Precision|              Recall|           F-Measure|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    1| 5.16051524648427E-6|1.115697869017070...|0.047619047619047616|1.115697869017070...|2.226179875333927E-4|
|    3|1.621647413230579...|2.797788852536087E-4|  0.3033240997229917|2.797788852536087E-4|5.590421234410958E-4|
|    5|0.001417628086974...|   0.086653150425235|  0.3763723696248856|   0.086653150425235| 0.14087280354429296|
|    4| 0.12462870623947307|  0.5495060044664929|  0.6418422928252964|  0.5495060044664929|  0.5920958541340119|
|    2|  0.5643995834662959|  0.9378193828723317|  0.6218636891968456|  0.9378193828723317|  0.7478388804458737|
|    0|                 0.0|                 0.0|                 0.0|                 0.0|     

In [67]:
// view metrics total
metrics_all(trainingSummary).show()

+-------------------+-------------------+
|             Metric|             Values|
+-------------------+-------------------+
|           Accuracy| 0.6261917418899768|
|False Positive Rate|0.31680498493928955|
| True Positive Rate| 0.6261917418899767|
|          F-Measure| 0.5445865731926203|
|          Precision| 0.5597212628084544|
|             Recall| 0.6261917418899767|
+-------------------+-------------------+



In [68]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Logistic Regression"), 
                                               metrics_all(trainingSummary).collect()(3)(1)
                                              ).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [69]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 41


## Random Forest

In [70]:
// define random forest model
val rf = new RandomForestClassifier()
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures1")
    .setNumTrees(50)
    .setSubsamplingRate(1)

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_177124423758


In [71]:
// train the model
val model = rf.fit(df_workingSet)

model: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_177124423758) with 50 trees


In [72]:
// get prediction values
val predictions = model.transform(df_workingSet)

predictions: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 13 more fields]


In [73]:
// compute metric
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_2323c1a81c3d


In [74]:
// get metric
val myMetric = evaluator.evaluate(predictions)

// view metric
println("F-Measure: " +  myMetric)

F-Measure: 0.3560933182077977


myMetric: Double = 0.3560933182077977


In [75]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Random Forest"), 
                                               myMetric).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [76]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 12
Current Minute: 47


## Multilayer Perceptron Classifier
A feedforward artificial neural network

**Specify layers for the neural network:**
  * input layer of size 201 (features), 
  * hidden layers of sizes 30 and 30,
  * output layer of size 7 (6 classes plus a 0 class)

In [77]:
// define layers
val layers = Array[Int](201, 30, 30, 7)

layers: Array[Int] = Array(201, 30, 30, 7)


In [78]:
// define the neural network model
val trainer = new MultilayerPerceptronClassifier()
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures1")
    .setLayers(layers)
    .setBlockSize(128)
    .setSeed(1234L)
    .setMaxIter(100)

trainer: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_7b54fc8143c4


In [79]:
// train the model - ignore executor memory warnings
val model = trainer.fit(df_workingSet)

model: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_7b54fc8143c4


In [80]:
// get prediction values
val predictions = model.transform(df_workingSet)

predictions: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 13 more fields]


In [81]:
// compute metric
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ae9c2f993240


In [82]:
// get metric
val myMetric = evaluator.evaluate(predictions)

// view metric
println("F-Measure: " +  myMetric)

F-Measure: 0.5985587381418374


myMetric: Double = 0.5985587381418374


In [83]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Multilayer Perceptron Classifier"), 
                                               myMetric).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [84]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 13
Current Minute: 8


## Decision Tree

In [85]:
// define the decision tree model
val dt = new DecisionTreeClassifier()
    .setLabelCol("label")
    .setFeaturesCol("comboFeatures1")

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_83860f050017


In [86]:
// train the model
val model = dt.fit(df_workingSet)

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_83860f050017) of depth 5 with 15 nodes


In [87]:
// get prediction values
val predictions = model.transform(df_workingSet)

predictions: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 13 more fields]


In [88]:
// compute metric
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_3bba9175ab5e


In [89]:
// get metric
val myMetric = evaluator.evaluate(predictions)

// view metric
println("F-Measure: " +  myMetric)

F-Measure: 0.6541996533779226


myMetric: Double = 0.6541996533779226


In [90]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Decision Tree"), 
                                               myMetric).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [91]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 13
Current Minute: 10


## Naive Bayes

In [92]:
// define the naive bayes model
val nb = new NaiveBayes()
    .setLabelCol("label")
    .setFeaturesCol("TF-IDF")

nb: org.apache.spark.ml.classification.NaiveBayes = nb_85e3632cf039


In [93]:
// train the model
val nbModel = nb.fit(df_workingSet)

nbModel: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel (uid=nb_85e3632cf039) with 6 classes


In [94]:
// get prediction values
val predictions = nbModel.transform(df_workingSet)

predictions: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 13 more fields]


In [95]:
// compute metric
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e13dee2431fd


In [96]:
// get metric
val myMetric = evaluator.evaluate(predictions)

// view metric
println("F-Measure: " +  myMetric)

F-Measure: 0.05874539353805046


myMetric: Double = 0.05874539353805046


In [97]:
//update results dataframe
results = results.withColumn("F-Measure", when(col("Model").equalTo("Naive Bayes"), 
                                               myMetric).otherwise(col("F-Measure")))

results: org.apache.spark.sql.DataFrame = [Model: string, F-Measure: double]


In [98]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 13
Current Minute: 11


### Model Testing Results

In [99]:
// view the model results
results.show(false)

+--------------------------------+-------------------+
|Model                           |F-Measure          |
+--------------------------------+-------------------+
|Logistic Regression             |0.5445865731926203 |
|Random Forest                   |0.3560933182077977 |
|Multilayer Perceptron Classifier|0.5985587381418374 |
|Decision Tree                   |0.6541996533779226 |
|Naive Bayes                     |0.05874539353805046|
+--------------------------------+-------------------+



* Decision Tree was the best performing model by more than 10% over the next best performing model, logistic regression  

**Decision Tree is therefore chosen as the most best performing model**

## Final Model: Hyperparameter Tuning
Hyperparameter tuning of the Decision Tree model using cross-validation and grid search of parameters

### Perform Parameter Search

In [100]:
// Define complete decision tree pipeline
val hashingTF2 = new HashingTF()
    .setInputCol("words_stopped")
    .setOutputCol("tf2")
var idf2 = new IDF().setInputCol("tf2").setOutputCol("idf")
val assembler2 = new VectorAssembler()
    .setInputCols(Array("parent_score", "idf"))
    .setOutputCol("comboFeaturesPipe")
val dt = new DecisionTreeClassifier()
    .setLabelCol("label")
    .setFeaturesCol("comboFeaturesPipe")
val pipeline = new Pipeline()
    .setStages(Array(hashingTF2, idf2,assembler2,dt))

hashingTF2: org.apache.spark.ml.feature.HashingTF = hashingTF_fcba16cf65b9
idf2: org.apache.spark.ml.feature.IDF = idf_bc0dced4da12
assembler2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_0521db7c142a
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_a176f0300ed1
pipeline: org.apache.spark.ml.Pipeline = pipeline_41f262bdf2b0


In [101]:
// define grid search parameters
val paramGrid = new ParamGridBuilder()
    .addGrid(hashingTF2.numFeatures, Array(200, 1000))
    .addGrid(dt.maxBins, Array(32,64))
    .addGrid(dt.maxDepth, Array(5,10))
    .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_a176f0300ed1-maxBins: 32,
	dtc_a176f0300ed1-maxDepth: 5,
	hashingTF_fcba16cf65b9-numFeatures: 200
}, {
	dtc_a176f0300ed1-maxBins: 32,
	dtc_a176f0300ed1-maxDepth: 5,
	hashingTF_fcba16cf65b9-numFeatures: 1000
}, {
	dtc_a176f0300ed1-maxBins: 64,
	dtc_a176f0300ed1-maxDepth: 5,
	hashingTF_fcba16cf65b9-numFeatures: 200
}, {
	dtc_a176f0300ed1-maxBins: 64,
	dtc_a176f0300ed1-maxDepth: 5,
	hashingTF_fcba16cf65b9-numFeatures: 1000
}, {
	dtc_a176f0300ed1-maxBins: 32,
	dtc_a176f0300ed1-maxDepth: 10,
	hashingTF_fcba16cf65b9-numFeatures: 200
}, {
	dtc_a176f0300ed1-maxBins: 32,
	dtc_a176f0300ed1-maxDepth: 10,
	hashingTF_fcba16cf65b9-numFeatures: 1000
}, {
	dtc_a176f0300ed1-maxBins: 64,
	dtc_a176f0300ed1-maxDepth: 10,
	hashingTF_fcba16c...

In [102]:
// define cross-validation parameters for the pipeline
val cv = new CrossValidator()
    .setEstimator(pipeline)
    .setEvaluator(new MulticlassClassificationEvaluator().setMetricName("f1"))
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3) 
    .setParallelism(2)

cv: org.apache.spark.ml.tuning.CrossValidator = cv_848afe09fef6


In [103]:
// Perform cross-validation for each set of grid search parameters - ignore executor memory warnings
val cvModel = cv.fit(df_workingSet)

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_848afe09fef6


In [104]:
// view the metric for each set of grid search parameters
cvModel.avgMetrics

res37: Array[Double] = Array(0.6465391863739914, 0.6478580969288755, 0.6540975626208687, 0.6554583318367874, 0.6491290332151539, 0.6543520828244637, 0.6516840512229796, 0.6547793771356328)


In [105]:
// get current time
println("Current Hour: " + LocalDateTime.now().getHour() + "\nCurrent Minute: " + LocalDateTime.now().getMinute())

Current Hour: 15
Current Minute: 46


#### Fit Decision Tree Model Using Best Parameters

In [106]:
// get prediction values using the best parameter set from the hyperparameter tuning
val predictions = cvModel.transform(df_workingSet)

predictions: org.apache.spark.sql.DataFrame = [id: string, parent_score: bigint ... 16 more fields]


In [107]:
// compute metric
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f2b9a1b793a6


In [108]:
// get metric
val myMetric = evaluator.evaluate(predictions)

// view metric
println("F-Measure: " +  myMetric)

F-Measure: 0.6558979125693123


myMetric: Double = 0.6558979125693123


### Retrieve Best Set of Parameters

In [109]:
// define variables of tuned parameters
val bestPipelineModel = cvModel.bestModel.asInstanceOf[PipelineModel]
val stages = bestPipelineModel.stages
val hashingStage = stages(0).asInstanceOf[HashingTF]
val dtStage = stages(3).asInstanceOf[DecisionTreeClassificationModel]

bestPipelineModel: org.apache.spark.ml.PipelineModel = pipeline_41f262bdf2b0
stages: Array[org.apache.spark.ml.Transformer] = Array(hashingTF_fcba16cf65b9, idf_bc0dced4da12, vecAssembler_0521db7c142a, DecisionTreeClassificationModel (uid=dtc_a176f0300ed1) of depth 5 with 23 nodes)
hashingStage: org.apache.spark.ml.feature.HashingTF = hashingTF_fcba16cf65b9
dtStage: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_a176f0300ed1) of depth 5 with 23 nodes


In [110]:
// view tuned parameters
println("Final selected decision tree hyperparameters:")
println("numFeatures = " + hashingStage.getNumFeatures)
println("MaxDepth = " + dtStage.getMaxDepth)
println("MaxBins = " + dtStage.getMaxBins)

Final selected decision tree hyperparameters:
numFeatures = 1000
MaxDepth = 5
MaxBins = 64


In [111]:
// view decision tree branching
dtStage.toDebugString

res41: String =
"DecisionTreeClassificationModel (uid=dtc_a176f0300ed1) of depth 5 with 23 nodes
  If (feature 0 <= 48.5)
   Predict: 3.0
  Else (feature 0 > 48.5)
   If (feature 0 <= 178.5)
    If (feature 0 <= 90.5)
     If (feature 0 <= 64.5)
      Predict: 3.0
     Else (feature 0 > 64.5)
      If (feature 667 <= 1.5735454947114331)
       Predict: 4.0
      Else (feature 667 > 1.5735454947114331)
       Predict: 3.0
    Else (feature 0 > 90.5)
     If (feature 0 <= 134.5)
      If (feature 667 <= 1.5735454947114331)
       Predict: 4.0
      Else (feature 667 > 1.5735454947114331)
       Predict: 3.0
     Else (feature 0 > 134.5)
      If (feature 667 <= 1.5735454947114331)
       Predict: 5.0
      Else (feature 667 > 1.5735454947114331)
       Predict: 3.0
   Else (feature 0 > 17...

#### Export Decision Tree branching using tuned parameters to file

In [112]:
// export decision tree to text file for review
new java.io.PrintWriter("./decisionTreeModel.txt") { write(dtStage.toDebugString); close }

res42: java.io.PrintWriter = $anon$1@39009fb3
